## Fuzzy Match POC with Apache Spark
The objective of this project is to test the execution of native spark functions to perform string similarity analysis, with variated similarity analysis algorithms

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import names
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

sc = SparkContext().getOrCreate()

spark = SparkSession.builder.appName(
    'Fuzzy Match POC').getOrCreate()

### Prepare datasets
`targets` is the name list that will be looked up inside `comparison`

In [2]:
vogals = ['a', 'e', 'i', 'o', 'u']
targets = []
comparison = []

for i in range(10):
    name = names.get_full_name()
    targets.append({'SimilarityWith': name})
    comparison.append({'Name': name})

    #replace vogals
    for vogal in vogals:
        for aux_vogal in vogals:
            if vogal != aux_vogal:
                shuffled_name = name.replace(vogal, aux_vogal)
                
                if shuffled_name not in list(map(lambda x: x['Name'], comparison)):
                    comparison.append({'Name': shuffled_name})

targets_df = spark.createDataFrame(targets).alias('t')
comparison_df = spark.createDataFrame(comparison).alias('c')

print('Targets to be searched:')
targets_df.show(n=2)

print('\Search list:')
comparison_df.show(n=2)

df = targets_df.crossJoin(comparison_df)

df = df.withColumn('Similarity', F.expr(
    'balogo_jaro_winkler(t.SimilarityWith, c.Name)'))

df.show()

Targets to be searched:
+-----------------+
|   SimilarityWith|
+-----------------+
|    Edward Hughes|
|Jermaine Harrison|
+-----------------+
only showing top 2 rows

\Search list:
+-------------+
|         Name|
+-------------+
|Edward Hughes|
|Edwerd Hughes|
+-------------+
only showing top 2 rows



AnalysisException: Undefined function: 'balogo_jaro_winkler'. This function is neither a registered temporary function nor a permanent function registered in the database 'default'.; line 1 pos 0